# Continuous Control

---

*Jun Zhu, zhujun981661@gmail.com, 08.2020*

In this notebook, I present the solution for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip install matplotlib

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from ddpg_agent import DdpgAgent
from utilities import check_environment, play, plot_score_history

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher_Linux/Reacher.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
brain_name, state_space, action_space = check_environment(env)
brain_name

### 3. Take Random Actions in the Environment

In [ ]:
# play(env, brain_name, continuous=True)

### 4. Train  a DDPG (deep deterministic policy gradient) agent

The DDPG algorithm proposed in this [paper](https://arxiv.org/pdf/1509.02971.pdf) was implemented. DDPG is an actor-critic, model-free algorithm based on the deterministic policy gradient that can operate over **continuous action space**.

In [ ]:
def init_layer(layer):
    layer.weight.data.uniform_(-3e-3, 3e-3)
    layer.bias.data.uniform_(-3e-3, 3e-3)


class Actor(nn.Module):
    def __init__(self):
        super().__init__()
        
        self._fc1 = nn.Linear(state_space, 512)
        self._fc2 = nn.Linear(512, 128)
        
        self._fc_final = nn.Linear(128, action_space)
        init_layer(self._fc_final)
        
    def forward(self, state):
        x = state
        for fc in (self._fc1, self._fc2):
            x = F.relu(fc(x))
        
        return torch.tanh(self._fc_final(x))
    

class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        
        self._fc1 = nn.Linear(state_space + action_space, 512)
        self._fc2 = nn.Linear(512, 32)
        
        self._fc_final = nn.Linear(32, 1)
        init_layer(self._fc_final)

    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        
        for fc in (self._fc1, self._fc2):
            x = F.relu(fc(x))
            
        return self._fc_final(x)

In [ ]:
target_score = 30

agent = DdpgAgent(action_space, (Actor(), Critic()), replay_memory_size=int(1e5))

scores = agent.train(env,
                     theta=0.15,
                     sigma=0.05,
                     tau=0.001,
                     gamma=0.99,
                     learning_rate=(1e-3, 1e-3),
                     weight_decay=(1.e-4, 1.e-4),
                     batch_size=128,
                     n_episodes=500,
                     target_score=target_score,
                     output_frequency=1)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(12, 6))
plot_score_history(ax, scores, target_score)

In [ ]:
for i in range(3):
    agent.play(env)

In [ ]:
env.close()

### 5. Further improvement